In [1]:
import os
import zipfile
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim

C:\Users\Srinivasan\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNModel, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 56 * 56, 512)  # Adjust input size based on your image dimensions
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        # Flatten before fully connected layers
        x = x.view(x.size(0), -1)

        # Fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)

        return x

In [3]:
class NewsDataset(Dataset):
    def __init__(self, text_file, image_folder, transform=None):
        self.text_data = pd.read_csv(text_file,encoding='latin-1')
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        text = self.text_data.iloc[idx]['hashvalue']
        
        # Check if 'img' column contains a valid string
        img_value = self.text_data.iloc[idx]['img']
        if pd.notna(img_value) and isinstance(img_value, str):
            image_path = os.path.join(self.image_folder, img_value)
            
            # Check if the image file exists
            if os.path.exists(image_path):
                image = Image.open(image_path).convert('RGB')
                if self.transform:
                    image = self.transform(image)
                return text, image
            else:
                # Handle missing image file
                # You can return a default image or skip the sample
                # Here, we return a default image (all zeros)
                default_image = Image.new('RGB', (224,224), (0, 0, 0))
                if self.transform:
                    default_image = self.transform(default_image)
                return text, default_image
        else:
            # Handle missing or invalid image path
            # You can return a default image or skip the sample
            # Here, we return a default image (all zeros)
            default_image = Image.new('RGB', (224,224), (0, 0, 0))
            if self.transform:
                default_image = self.transform(default_image)
            return text, default_image



In [4]:
# Set up data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [5]:
# Initialize your model, dataset, and dataloaders
model = CNNModel()
dataset = NewsDataset(text_file='rt-train-textImg.csv', image_folder='images-Train', transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [6]:
# Inside your training loop
k_values = [1, 5, 10, 20, 50, 100]
precision_at_k = {k: 0 for k in k_values}
num_epochs=10
for epoch in range(num_epochs):
    total_correct = 0
    total_samples = 0

    for text, image in train_loader:
        labels = torch.randint(0, 2, (len(text),))
        optimizer.zero_grad()
        outputs = model(image)
        
        # Convert outputs to probabilities and get predicted labels
        probabilities = torch.softmax(outputs, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)

        # Compute accuracy
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += len(labels)

        # Calculate Precision@K
        for k in k_values:
            top_k_predictions = predicted_labels.argsort(descending=True)[:k]
            precision_at_k[k] += (labels[top_k_predictions] == 1).sum().item() / k

        # Calculate loss and perform backpropagation as before

    accuracy = total_correct / total_samples
    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.4f}')

# Calculate and print Precision@K
for k, value in precision_at_k.items():
    precision = value / total_samples
    print(f'Precision@{k}: {precision:.4f}')

Epoch [1/10], Accuracy: 0.5010
Epoch [2/10], Accuracy: 0.4914
Epoch [3/10], Accuracy: 0.5037
Epoch [4/10], Accuracy: 0.4910
Epoch [5/10], Accuracy: 0.5008
Epoch [6/10], Accuracy: 0.5079
Epoch [7/10], Accuracy: 0.4944
Epoch [8/10], Accuracy: 0.5006
Epoch [9/10], Accuracy: 0.5056
Epoch [10/10], Accuracy: 0.4994
Precision@1: 0.1582
Precision@5: 0.1561
Precision@10: 0.1551
Precision@20: 0.1559
Precision@50: 0.1001
Precision@100: 0.0500


In [7]:
import torch

# Inside your training loop
reciprocal_ranks = []
num_epochs=10
for epoch in range(num_epochs):
    total_correct = 0
    total_samples = 0

    for text, image in train_loader:
        labels = torch.randint(0, 2, (len(text),))
        optimizer.zero_grad()
        outputs = model(image)
        
        # Convert outputs to probabilities and get predicted labels
        probabilities = torch.softmax(outputs, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)

        # Compute accuracy
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += len(labels)

        # Calculate reciprocal rank
        for idx, label in enumerate(labels):
            if label == 1:
                # The reciprocal rank is 1 / (position + 1)
                reciprocal_rank = 1 / (predicted_labels[idx].item() + 1)
                reciprocal_ranks.append(reciprocal_rank)

        # Calculate loss and perform backpropagation as before

    accuracy = total_correct / total_samples
    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.4f}')

# Calculate and print Mean Reciprocal Rank (MRR)
mrr = sum(reciprocal_ranks) / len(reciprocal_ranks)
print(f'Mean Reciprocal Rank (MRR): {mrr:.4f}')


Epoch [1/10], Accuracy: 0.4985
Epoch [2/10], Accuracy: 0.4921
Epoch [3/10], Accuracy: 0.4936
Epoch [4/10], Accuracy: 0.5046
Epoch [5/10], Accuracy: 0.5022
Epoch [6/10], Accuracy: 0.4999
Epoch [7/10], Accuracy: 0.4968
Epoch [8/10], Accuracy: 0.4998
Epoch [9/10], Accuracy: 0.4996
Epoch [10/10], Accuracy: 0.4945
Mean Reciprocal Rank (MRR): 1.0000


In [34]:
def get_prediction(model, images):
    # Preprocess images if needed
    # ...

    # Convert images to torch tensors
    # Assuming 'images' is a list of PIL images
    tensor_images = [transforms.ToTensor()(img) if not isinstance(img, torch.Tensor) else img for img in images]
    input_tensor = torch.stack(tensor_images)

    # Forward pass through the model
    with torch.no_grad():
        model.eval()
        outputs = model(input_tensor)

    # Assuming a classification task where you want to get class probabilities
    probabilities = nn.functional.softmax(outputs, dim=1)

    # Get the predicted class or classes (based on your task)
    _, predicted_classes = torch.max(probabilities, 1)

    # Convert predictions to a list of strings
    predictions = [str(pred.item()) for pred in predicted_classes]

    return predictions


In [35]:
# Initialize your model, dataset, and dataloaders
model = CNNModel()
test_dataset = NewsDataset(text_file="C:/Users/Srinivasan/Desktop/NewsImages/RT-Dataset/rt-test-text-1.csv", image_folder='images-Test', transform=transform)
test_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [46]:
def get_predictions(model, images):
    # Unpack the tuples to get the image paths
    image_paths = [img[1] for img in images]

    # Assuming 'image_paths' is a list of PIL images or tensors
    tensor_images = [img if isinstance(img, torch.Tensor) else transforms.ToTensor()(img) for img in image_paths]
    input_tensor = torch.stack(tensor_images)

    # Assuming your model is already in evaluation mode
    model.eval()

    # Forward pass
    with torch.no_grad():
        outputs = model(input_tensor)

    # Assuming 'get_prediction' takes model outputs as an argument
    predictions = get_prediction(model, outputs, image_paths)

    return predictions



In [47]:
def predict_and_format_output(model, data_loader):
    all_predictions = []

    for text, image_paths in data_loader:
        # Assuming you have a function to get predictions for each image
        predictions = get_predictions(model, image_paths)

        # Format the output for each news URL
        news_url = text[0]  # Assuming there is one URL per batch
        formatted_output = f"{news_url}\t{''.join(predictions)}"

        all_predictions.append(formatted_output)

    return all_predictions


# Usage
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
predictions = predict_and_format_output(model, test_loader)

# Save predictions to a file
with open('predictions.txt', 'w') as f:
    f.write('\n'.join(predictions))


RuntimeError: Given groups=1, weight of size [16, 3, 3, 3], expected input[1, 1, 224, 224] to have 3 channels, but got 1 channels instead